In [1]:
import pandas as pd

data = pd.read_csv('games.csv', index_col=0)

In [2]:
# Cleaning / getting data ready for machine learning
data['venue'] = data['venue'].map({'Home' : 1, 'Away' : 0})     # convert venue to 1's and 0's

In [3]:
data

,Date,Time,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,7:00 PM,Anaheim Ducks,St. Louis Blues,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,10:30 PM,Anaheim Ducks,Arizona Coyotes,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,8:00 PM,Anaheim Ducks,Minnesota Wild,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,10:00 PM,Anaheim Ducks,Vancouver Canucks,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,10:00 PM,Anaheim Ducks,Seattle Kraken,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,Tampa Bay Lightning,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,7:00 PM,Winnipeg Jets,New Jersey Devils,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,7:00 PM,Winnipeg Jets,New York Rangers,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,5:00 PM,Winnipeg Jets,Carolina Hurricanes,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [4]:
data['Opponent'] = data['Opponent'].astype('category').cat.codes      # opponent is now represented by a number
data

,Date,Time,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,7:00 PM,Anaheim Ducks,25,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,10:30 PM,Anaheim Ducks,1,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,8:00 PM,Anaheim Ducks,14,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,10:00 PM,Anaheim Ducks,29,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,10:00 PM,Anaheim Ducks,24,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,26,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,7:00 PM,Winnipeg Jets,17,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,7:00 PM,Winnipeg Jets,19,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,5:00 PM,Winnipeg Jets,5,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [5]:
games = data.drop(columns=['Time'], inplace=False)
games

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,Anaheim Ducks,25,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,Anaheim Ducks,1,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,Anaheim Ducks,14,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,Anaheim Ducks,29,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,Anaheim Ducks,24,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,Winnipeg Jets,26,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,Winnipeg Jets,17,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,Winnipeg Jets,19,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,Winnipeg Jets,5,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [6]:
# Dropping data for Arizona Coyotes (not a team anymore)
games = games[games["Team"] != "Arizona Coyotes"]

In [7]:
training = games[games['Date'] < '2024-04-19']  # Training using 2021-2024 data
testing = games[games['Date'] > '2024-04-19']   # Testing on most recent season (2024-2025)

import xgboost as xgb

In [8]:
# Dropping 'Date' column because not needed anymore
training = training.drop(columns=['Date'])
testing = testing.drop(columns=['Date'])


In [9]:
# Getting training setup
x_train = training.drop(columns=['Result'])
y_train = training['Result']

# Getting testing setup
x_test = testing.drop(columns=['Result'])
y_test = testing['Result']

In [10]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=10)

In [11]:
# Defining search space for GridSearchCV
search_grid = {
    'n_estimators' : [50, 100, 200, 500],
    'max_depth' : [3, 6, 9],
    'learning_rate' : [0.001, 0.01, 0.1],
    'reg_alpha': [0, 0.1, 1, 5, 10],
    'reg_lambda': [0.1, 1, 5, 10, 20],

}

In [12]:
games

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,Anaheim Ducks,25,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,Anaheim Ducks,1,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,Anaheim Ducks,14,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,Anaheim Ducks,29,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,Anaheim Ducks,24,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,Winnipeg Jets,26,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,Winnipeg Jets,17,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,Winnipeg Jets,19,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,Winnipeg Jets,5,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [13]:
# Computing rolling averages \
games_o = games.groupby("Team")
team = games_o.get_group("Anaheim Ducks").sort_values("Date")

def rolling_averages(team, cols, new_cols):
    team = team.sort_values("Date")    # Getting team data organized chronologically
    rolling = team[cols].rolling(3, closed='left').mean()   # closed=left to ignore current row in sliding window
    team[new_cols] = rolling
    team = team.dropna(subset=new_cols) # dropping first rows because not enough data
    return team


In [14]:
team

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
293,2021-10-13,Anaheim Ducks,32,1,16260.0,4,1,22,18.2,0.971,24,34,2.9,0.818,14,1
270,2021-10-15,Anaheim Ducks,14,1,11938.0,1,2,29,3.4,0.953,25,43,4.7,0.966,23,0
140,2021-10-18,Anaheim Ducks,4,0,15174.0,3,2,27,11.1,0.953,11,43,4.7,0.889,9,1
121,2021-10-19,Anaheim Ducks,11,0,14082.0,5,6,36,13.9,0.861,12,37,16.2,0.733,4,0
103,2021-10-21,Anaheim Ducks,32,0,13886.0,1,5,39,2.6,0.846,4,27,18.5,0.974,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2025-04-09,Anaheim Ducks,4,1,15134.0,4,3,23,17.4,0.923,12,39,7.7,0.826,2,1
50,2025-04-10,Anaheim Ducks,13,0,18145.0,1,6,25,4.0,0.793,14,29,20.7,0.960,10,0
194,2025-04-13,Anaheim Ducks,7,1,17241.0,2,4,21,9.5,0.875,6,25,16.0,0.905,4,0
234,2025-04-15,Anaheim Ducks,14,0,19029.0,2,3,29,6.9,0.925,4,40,7.5,0.917,2,0


In [15]:
cols = ['G', 'GA', 'S', 'S%', 'SV%', 'PIM', 'Result']   # wanted columns for rolling
new_cols = [f"{c}_rolling" for c in cols]

In [16]:
games_data = games.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols))
games_data = games_data.droplevel("Team")
games_data

C:\Users\vikto\AppData\Local\Temp\ipykernel_25552\700468304.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_data = games.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols))


,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,Opponent SV%,Opponent PIM,Result,G_rolling,GA_rolling,S_rolling,S%_rolling,SV%_rolling,PIM_rolling,Result_rolling
121,2021-10-19,Anaheim Ducks,11,0,14082.0,5,6,36,13.9,0.861,...,0.733,4,0,2.666667,1.666667,26.000000,10.900000,0.959000,20.000000,0.666667
103,2021-10-21,Anaheim Ducks,32,0,13886.0,1,5,39,2.6,0.846,...,0.974,6,0,3.000000,3.333333,30.666667,9.466667,0.922333,16.000000,0.333333
104,2021-10-23,Anaheim Ducks,14,0,18055.0,3,4,24,12.5,0.889,...,0.875,10,0,3.000000,4.333333,34.000000,9.200000,0.886667,9.000000,0.333333
257,2021-10-26,Anaheim Ducks,32,1,11951.0,3,4,35,8.6,0.840,...,0.914,6,0,3.000000,5.000000,33.000000,9.666667,0.865333,9.333333,0.000000
228,2021-10-28,Anaheim Ducks,3,1,12014.0,3,4,37,8.1,0.862,...,0.919,2,0,2.333333,4.333333,32.666667,7.900000,0.858333,6.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10292,2025-04-07,Winnipeg Jets,25,1,15225.0,3,1,26,11.5,0.933,...,0.920,2,1,2.000000,2.666667,24.666667,8.833333,0.910333,3.333333,0.333333
10219,2025-04-10,Winnipeg Jets,9,0,18532.0,4,0,35,11.4,1.000,...,0.886,7,1,2.666667,1.666667,27.000000,10.900000,0.940667,4.000000,0.666667
10208,2025-04-12,Winnipeg Jets,6,0,20634.0,5,4,42,9.5,0.875,...,0.905,4,1,2.666667,1.666667,31.333333,8.633333,0.940667,5.666667,0.666667
10251,2025-04-13,Winnipeg Jets,11,1,15225.0,1,4,18,5.6,0.921,...,0.944,4,0,4.000000,1.666667,34.333333,10.800000,0.936000,5.000000,1.000000


In [17]:
# Making sure number of games dropped add up
# Dropped 3 games for 33 teams (32 + Utah Hockey club added last season). Expecting 99
print(f"Number of rows before: {games.shape[0]}")
print(f"Number of rows after: {games_data.shape[0]}")
print(f"Rows removed: {(games.shape[0]) - (games_data.shape[0])}")

Number of rows before: 10250
Number of rows after: 10151
Rows removed: 99


In [18]:
games_data.index = range(games_data.shape[0])  # fixing index level
games_data

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,Opponent SV%,Opponent PIM,Result,G_rolling,GA_rolling,S_rolling,S%_rolling,SV%_rolling,PIM_rolling,Result_rolling
0,2021-10-19,Anaheim Ducks,11,0,14082.0,5,6,36,13.9,0.861,...,0.733,4,0,2.666667,1.666667,26.000000,10.900000,0.959000,20.000000,0.666667
1,2021-10-21,Anaheim Ducks,32,0,13886.0,1,5,39,2.6,0.846,...,0.974,6,0,3.000000,3.333333,30.666667,9.466667,0.922333,16.000000,0.333333
2,2021-10-23,Anaheim Ducks,14,0,18055.0,3,4,24,12.5,0.889,...,0.875,10,0,3.000000,4.333333,34.000000,9.200000,0.886667,9.000000,0.333333
3,2021-10-26,Anaheim Ducks,32,1,11951.0,3,4,35,8.6,0.840,...,0.914,6,0,3.000000,5.000000,33.000000,9.666667,0.865333,9.333333,0.000000
4,2021-10-28,Anaheim Ducks,3,1,12014.0,3,4,37,8.1,0.862,...,0.919,2,0,2.333333,4.333333,32.666667,7.900000,0.858333,6.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10146,2025-04-07,Winnipeg Jets,25,1,15225.0,3,1,26,11.5,0.933,...,0.920,2,1,2.000000,2.666667,24.666667,8.833333,0.910333,3.333333,0.333333
10147,2025-04-10,Winnipeg Jets,9,0,18532.0,4,0,35,11.4,1.000,...,0.886,7,1,2.666667,1.666667,27.000000,10.900000,0.940667,4.000000,0.666667
10148,2025-04-12,Winnipeg Jets,6,0,20634.0,5,4,42,9.5,0.875,...,0.905,4,1,2.666667,1.666667,31.333333,8.633333,0.940667,5.666667,0.666667
10149,2025-04-13,Winnipeg Jets,11,1,15225.0,1,4,18,5.6,0.921,...,0.944,4,0,4.000000,1.666667,34.333333,10.800000,0.936000,5.000000,1.000000


In [24]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(
    estimator = model,
    param_grid = search_grid,
    scoring = ["accuracy"],
    refit = "accuracy",
    cv = 5,
    verbose= 2
)

In [25]:
GS.fit(x_train, y_train)

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=0.1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=0.1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=0.1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=0.1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=0.1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_lambda=1; total time=   0.0s
[CV] END learning_rate=0.001, max_depth=3, n_estimators=50, reg_alpha=0, reg_l

ValueError: 
All the 4500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4500 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vikto\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\data.py", line 407, in pandas_feature_info
    new_feature_types.append(_pandas_dtype_mapper[dtype.name])
                             ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\vikto\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vikto\AppData\Local\Programs\Python\Python312